In [1]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm



import numpy as np
import os
import shutil

import tensorflow as tf

from tensorflow import keras

# from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import optimizers, initializers, regularizers, metrics
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10960321526693127130
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4986830848
locality {
  bus_id: 1
  links {
  }
}
incarnation: 461723751756343274
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [3]:
from io import BytesIO
import time, random
from tqdm import tqdm

from PIL import Image

In [4]:
BASE_DIR = "D:/original_image/"
names =  ["63building", "castle","coex","general", "indep_door","judgement_castle","lotte_tower","moonlight",\
                "namsan", "seoulstation","tapgol_park"]

tf.random.set_seed(1)

In [5]:
if not os.path.isdir(BASE_DIR + 'train/'):
    for name in names:
        os.makedirs(BASE_DIR + 'train/' + name)
        os.makedirs(BASE_DIR + 'val/' + name)
        os.makedirs(BASE_DIR + 'test/' + name)
        

In [6]:
org_folders =  ["63building/", "castle/","coex/","general/", "indep_door/","judgement_castle/","lotte_tower/","moonlight/",\
                "namsan/", "seoulstation/","tapgol_park/"]

In [7]:
# for folder_idx, folder in enumerate(orig_folders):
#     files = os.listdir(BASE_DIR + folder)
#     number_of_images = len([name for name in files])
#     n_train = int((number_of_images * 0.6) + 0.5)
#     n_valid = int((number_of_images* 0.25) + 0.5)
#     n_test = number_of_images - n_train - n_valid
#     print(number_of_images, n_train, n_valid, n_test)
#     for idx, file in enumerate(files):
#         file_name = BASE_DIR + folder + file
#         if idx < n_train:
#             shutil.move(file_name, BASE_DIR + "train/" + names[folder_idx])
#         elif idx < n_train + n_valid:
#             shutil.move(file_name, BASE_DIR + "val/" + names[folder_idx])
#         else:
#             shutil.move(file_name, BASE_DIR + "test/" + names[folder_idx])

In [8]:
for folder_idx, folder in enumerate(org_folders):
    sample_list = [x for x in os.listdir(BASE_DIR + folder)]
    
    not_train_list = random.sample(sample_list, int(len(sample_list)*0.4))
    train_list = [x for x in sample_list if x not in not_train_list]
    validation_list = random.sample(not_train_list, int(len(not_train_list)/2))
    test_list = [x for x in not_train_list if x not in validation_list]
    

    for file in tqdm(test_list):
        image = Image.open(BASE_DIR + folder + file)
        image.save(BASE_DIR + "test/"+ folder + file)

    for file in tqdm(validation_list):
        image = Image.open(BASE_DIR + folder + file)
        image.save(BASE_DIR + "val/"+ folder + file)

    for file in tqdm(train_list):
        image = Image.open(BASE_DIR + folder + file)
        image.save(BASE_DIR + "train/"+ folder + file)

  0%|▍                                                                                 | 1/200 [00:00<00:09, 20.85it/s]


KeyboardInterrupt: 

In [ ]:
train_gen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
valid_gen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_gen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
final_test_gen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

#train에 저장
train_batches = train_gen.flow_from_directory(
    'D:/original_image/train',
    target_size = (240, 240),
    class_mode = 'sparse',
    batch_size = 2,
    shuffle=True,
    color_mode="rgb",
    classes=names
)

#val에 저장
val_batches = valid_gen.flow_from_directory(
    'D:/original_image/val',
    target_size = (240, 240),
    class_mode = 'sparse',
    batch_size = 2,
    shuffle=False,
    color_mode="rgb",
    classes=names
)

#테스트에 저장
test_batches = test_gen.flow_from_directory(
    'D:/original_image/test',
    target_size = (240, 240),
    class_mode = 'sparse',
    batch_size = 2,
    shuffle=False,
    color_mode="rgb",
    classes=names
)

# 최종 테스트
final_test_batches = final_test_gen.flow_from_directory(
    'D:/sub_final_test/',
    target_size = (240, 240),
    class_mode = 'sparse',
    batch_size = 2,
    shuffle=False,
    color_mode="rgb"
)

In [ ]:
model = keras.models.Sequential()
model.add(layers.Conv2D(32, (3,3), strides=(1,1), padding="valid", activation='relu', input_shape=(240,240,3)))
model.add(layers.MaxPool2D(2,2))

In [ ]:
train_batch = train_batches[0]
print(train_batch[0].shape)
print(train_batch[1])

test_batch = test_batches[0]
print(test_batch[0].shape)
print(test_batch[1])

In [ ]:
# vgg 모델
vgg_model = tf.keras.applications.vgg16.VGG16()
print(type(vgg_model))
vgg_model.summary()

In [ ]:
model = tf.keras.models.Sequential()
for layer in vgg_model.layers[0:-1]:
    model.add(layer)

In [ ]:
model.summary()

In [ ]:
# 마지막 레이어만 train해주면됨
#앞에 레이어 non trainable로 바꿔줌
for layer in model.layers:
    layer.trainable = False
model.summary()

In [ ]:
model.add(layers.Dense(11))

In [ ]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim = keras.optimizers.Adam(lr=0.001)
metrics = ["accuracy"]

model.compile(optimizer=optim, loss=loss, metrics=metrics)

In [ ]:
preprocess_input = tf.keras.applications.vgg16.preprocess_input

In [ ]:
train_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
valid_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
test_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
final_test_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

train_batches = train_gen.flow_from_directory(
    'D:/original_image/train',
    target_size = (240, 240),
    class_mode = 'sparse',
    batch_size = 2,
    shuffle=True,
    color_mode="rgb",
    classes=names
)

val_batches = valid_gen.flow_from_directory(
    'D:/original_image/val',
    target_size = (240, 240),
    class_mode = 'sparse',
    batch_size = 2,
    shuffle=False,
    color_mode="rgb",
    classes=names
)

test_batches = test_gen.flow_from_directory(
    'D:/original_image/test',
    target_size = (240, 240),
    class_mode = 'sparse',
    batch_size = 2,
    shuffle=False,
    color_mode="rgb",
    classes=names
)

final_test_batches = final_test_gen.flow_from_directory(
    'D:/sub_final_test/',
    target_size = (240, 240),
    class_mode = 'sparse',
    batch_size = 2,
    shuffle=False,
    color_mode="rgb"
)

In [ ]:
epochs = 15

early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5
)

history = model.fit(train_batches, validation_data=val_batches,
         callbacks=[early_stopping],
         epochs=epochs)

In [ ]:
loss, acc = model.evaluate(test_batches)
acc

In [ ]:
model.save('model')

In [ ]:
new_model = load_model('model')

In [ ]:
new_model.evaluate(final_test_batches)

In [ ]:
# datagen = ImageDataGenerator(
#         rotation_range=2,
#         width_shift_range=0.05,
#         height_shift_range=0.05,
#         shear_range=0.05,
#         zoom_range = 0.1,
#         horizontal_flip=False,
#         fill_mode="nearest")

# folder = "D:/sub_final_test"

# file_list = [x for x in os.listdir(folder)]

# # sub_folder = "D:/final_test/" + "63buliding"
# # sub_file_list = [x for x in os.listdir(sub_folder)]

# for i in tqdm(range(len(file_list))):
#     sub_folder = "D:/sub_final_test/" + file_list[i]
#     sub_file_list = [x for x in os.listdir(sub_folder)]

#     for j in range(len(sub_file_list)):
#         image = load_img("D:/sub_final_test/" + file_list[i] +"/" + sub_file_list[j])  # PIL 이미지
#         x = img_to_array(image)  # (3, 300, 300) 크기의 NumPy 배열
#         x = x.reshape((1,) + x.shape)  # (1, 3, 300, 300) 크기의 NumPy 배열

#     #     아래 .flow() 함수는 임의 변환된 이미지를 배치 단위로 생성해서
#     #     지정된 `preview/` 폴더에 저장합니다.
#         k = 0
#         for batch in datagen.flow(x, batch_size=5,
#                                   save_to_dir="D:/sub_final_test/" + file_list[i], save_prefix="super_" + file_list[i], save_format="jpg"):
#             k += 1
#             time.sleep(0.01)
#             if k > 18:
                break  # 이미지 50장을 생성하고 마칩니다

In [ ]:
prediction = new_model.predict(final_test_batches)

In [ ]:
import glob

In [ ]:
caltech_dir = "D:/final_test"


X = []
filenames = []
files = glob.glob(caltech_dir+"/*.*")
for i, f in enumerate(files):
    img = Image.open(f)
    img = img.convert("RGB")
    data = np.asarray(img)
    filenames.append(f)
    X.append(data)

X = np.array(X)

In [ ]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
cnt = 0

In [ ]:
new_prediction = new_model.predict(X)

In [ ]:
final_names = ["63building", "castle","coex","general", "indep_door","judgement_castle","lotte_tower","moonlight",\
                "namsan", "seoulstation","tapgol_park"]

cnt=0
right_answer=0
error=0

# 이 비교는 그냥 파일들이 있으면 해당 파일과 비교. 카테고리와 함께 비교해서 진행하는 것은 _4 파일.
for i in new_prediction:
    pre_ans = i.argmax()  # 예측 레이블
    pre_ans_str = ''
    
    if pre_ans == 0: pre_ans_str = "63buliding"
    elif pre_ans == 1: pre_ans_str = "castle"
    elif pre_ans == 2: pre_ans_str = "coex"
    elif pre_ans == 3: pre_ans_str = "general"
    elif pre_ans == 4: pre_ans_str = "indep_door"
    elif pre_ans == 5: pre_ans_str = "judgement_castle"
    elif pre_ans == 6: pre_ans_str = "lotte_tower"
    elif pre_ans == 7: pre_ans_str = "moonlight"
    elif pre_ans == 8: pre_ans_str = "namsan"
    elif pre_ans == 9: pre_ans_str = "seoulstation"
    elif pre_ans == 10: pre_ans_str = "tapgol_park"
        
        
    else: pre_ans_str = "what the!!"
        

#     if i[0] >= 0.8 : print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
#     if i[1] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"으로 추정됩니다.")
#     if i[2] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
#     if i[3] >= 0.8 : print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
#     if i[4] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"으로 추정됩니다.")
#     if i[5] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
#     if i[6] >= 0.8 : print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
#     if i[7] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"으로 추정됩니다.")
#     if i[8] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
#     if i[9] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"으로 추정됩니다.")
#     if i[10] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")

    print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    cnt += 1
    
    try:        
        if pre_ans_str in filenames[cnt].split("\\")[1]:
            right_answer+=1
        else:
            error+=1
    except:
        pass

    #얘가 레이블 [1. 0. 0.] 이런식으로 되어 있는 것을 숫자로 바꿔주는 것.
#     즉 얘랑, 나중에 카테고리 데이터 불러와서 카테고리랑 비교를 해서 같으면 맞는거고, 아니면 틀린거로 취급하면 된다.
#     이걸 한 것은 _4.py에.

percent=round((right_answer/(right_answer + error)),4)
print(percent)
print(right_answer+error)
print(len(new_prediction))

In [ ]:
new_prediction[0]

In [12]:
saved_model_dir = 'model'
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()
open('converter/converted_model.tflite', 'wb').write(tflite_model)







537231084